# Asset Allocation using Deep Reinforcement Learning
----

## Data preprocess
---
* Data is Downloaded from Yahoo Finance using an API in the FinRL Library
* The 20 tickers in US stock for the period from 2006 to 2020 are used
* The data is downloaded and the cleaned data are saved in pkl files for use

---


### 1 Import Relevant Packages

In [1]:
import pandas as pd
import numpy as np
import pickle
from config import config
import matplotlib.pylab as plt
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
%matplotlib inline

### 2 Create Folders

In [3]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

### 3. Define 20 stock tickers

In [2]:
ticker_list = config.STOCK_TICKER
ticker_list

['AAPL',
 'MSFT',
 'JPM',
 'XOM',
 'RTX',
 'PG',
 'GS',
 'NKE',
 'BAC',
 'JNJ',
 'CVX',
 'AMZN',
 'GOOG',
 'PFE',
 'BA',
 'IBM',
 'CSCO',
 'AXP',
 'HD',
 'INTC']

In [3]:
#Download the data
df = pdr.get_data_yahoo([ticker_list][0], 
                          start='2006-01-01', end="2021-01-01")

[*********************100%***********************]  20 of 20 completed


In [4]:
data = df.copy()

In [5]:
data = data.stack().reset_index()
data.columns.names = [None]
data = data.drop(['Close'], axis=1)

In [6]:
data.head()

,Date,level_1,Adj Close,High,Low,Open,Volume
0,2006-01-03,AAPL,2.272182,2.669643,2.580357,2.585000,807234400
1,2006-01-03,AMZN,2.379000,2.392500,2.312500,2.373500,151644000
2,2006-01-03,AXP,40.282257,52.580002,51.049999,51.700001,7825700
3,2006-01-03,BA,50.119690,70.599998,69.330002,70.400002,4943000
4,2006-01-03,BAC,32.956573,47.180000,46.150002,46.919998,16296700


In [7]:
data.columns = ['date','tic','close','high','low','open','volume']

In [8]:
data.columns

Index(['date', 'tic', 'close', 'high', 'low', 'open', 'volume'], dtype='object')

### 4. Clean the Data

In [9]:
# check for missing values
data.isna().sum()

date      0
tic       0
close     0
high      0
low       0
open      0
volume    0
dtype: int64

In [10]:
date_set = data[data['tic']==ticker_list[0]].date.reset_index(drop=True)
date_set

0      2006-01-03
1      2006-01-04
2      2006-01-05
3      2006-01-06
4      2006-01-09
          ...    
3771   2020-12-24
3772   2020-12-28
3773   2020-12-29
3774   2020-12-30
3775   2020-12-31
Name: date, Length: 3776, dtype: datetime64[ns]

In [11]:
# check if all tickers have the same number of data points
no_datasets = []
for i in ticker_list:
    date_set_i = data[data['tic']==i].date.reset_index(drop=True)
    if not date_set_i.equals(date_set):
        print("Different dateset!!!")
        print(date_set_i)
    no_data_points = data[data['tic']==i].shape[0]
    no_datasets.append((i,no_data_points))
    data_points_df = pd.DataFrame(no_datasets)

In [12]:
data_points_df

,0,1
0,AAPL,3776
1,MSFT,3776
2,JPM,3776
3,XOM,3776
4,RTX,3776
5,PG,3776
6,GS,3776
7,NKE,3776
8,BAC,3776
9,JNJ,3776


In [13]:
data.head()

,date,tic,close,high,low,open,volume
0,2006-01-03,AAPL,2.272182,2.669643,2.580357,2.585000,807234400
1,2006-01-03,AMZN,2.379000,2.392500,2.312500,2.373500,151644000
2,2006-01-03,AXP,40.282257,52.580002,51.049999,51.700001,7825700
3,2006-01-03,BA,50.119690,70.599998,69.330002,70.400002,4943000
4,2006-01-03,BAC,32.956573,47.180000,46.150002,46.919998,16296700


In [43]:
data.to_csv('datasets/data.csv', index=False)

### 5. Create Covariance feature

In [14]:
def add_cov_matrix(df):
    """
    Function to add Coveriance Matrices as part of the defined states
    """
    # Sort the data and index by date and tic
    df=df.sort_values(['date','tic'],ignore_index=True) 
    df.index = df.date.factorize()[0]
    
    cov_list = [] # create empty list for storing coveriance matrices at each time step
    
    # look back for constructing the coveriance matrix is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values 
        covs = covs#/covs.max()
        cov_list.append(covs)
        
    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    
    return df

In [15]:
data_with_covs = data.copy()
data_with_covs = add_cov_matrix(data_with_covs)

In [ ]:
with open('datasets/data_ftr.pkl', 'wb') as handle:
    pickle.dump(data_with_covs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 6 Create a Dataset for the Close Prices to calculate return

In [59]:
# read the data from the saved csv file
# df_prices = pd.read_csv('./datasets/data.csv')
df_prices = data

In [32]:
# Reset the Index to tic and date
df_prices = df_prices.reset_index().set_index(['tic', 'date']).sort_index()

# Get the list of all the tickers
tic_list = list(set([i for i,j in df_prices.index]))

# Create an empty data frame for the close prices
df_close = pd.DataFrame()

In [33]:
len(tic_list)

20

In [34]:
# Reset the Index to tic and date
df_prices = df_prices.reset_index().set_index(['tic', 'date']).sort_index()

# Get all the Close Prices
df_close = pd.DataFrame()

for ticker in tic_list:
    series = df_prices.xs(ticker).close
    df_close[ticker] = series
    
df_close = df_close.reset_index()

In [35]:
df_close.head()

,date,JNJ,XOM,AAPL,IBM,CVX,GS,BAC,BA,RTX,...,CSCO,JPM,AXP,AMZN,INTC,NKE,HD,PG,GOOG,PFE
0,2006-01-03,37.529842,32.591087,2.272182,46.662437,30.822956,100.474739,32.956562,50.119694,23.914030,...,12.293545,25.504070,40.282272,2.3790,15.344927,8.689151,26.792191,35.971073,10.840119,11.357101
1,2006-01-04,38.108356,32.646820,2.278869,46.599873,30.734259,99.086967,32.606575,50.639107,23.770205,...,12.575343,25.356871,39.890656,2.3625,15.548970,8.648710,26.298439,36.038387,11.089434,11.724845
2,2006-01-05,37.950035,32.485184,2.260935,46.912624,30.358639,99.047974,32.648540,50.041435,23.681366,...,12.927595,25.433683,40.312973,2.3825,15.765004,8.701281,26.038576,35.922104,11.238874,11.739172
3,2006-01-06,38.120525,33.126194,2.319297,48.305786,30.911657,100.451370,32.599552,49.344135,23.757509,...,13.223483,25.612875,40.451199,2.3935,15.789012,8.673985,26.233480,35.885387,11.598028,11.868119
4,2006-01-09,38.358036,33.109467,2.311697,47.612072,30.755129,101.659805,32.620548,48.931450,24.028240,...,13.427788,26.028873,41.457088,2.3540,15.885030,8.795300,26.519325,36.154640,11.628912,11.868119


In [36]:
# Get Discriptive statistics
df_close.describe().T

,count,mean,std,min,25%,50%,75%,max
JNJ,3776.0,73.727707,34.016555,30.898920,42.528811,66.969944,105.661104,148.513046
XOM,3776.0,52.519638,10.136800,26.564503,43.539031,54.557209,60.839462,70.642685
AAPL,3776.0,24.228471,24.525643,1.540220,5.789124,17.123212,33.270695,134.862686
IBM,3776.0,96.346891,25.481514,42.091377,74.885229,105.559914,115.430029,138.501801
CVX,3776.0,67.805086,20.943232,28.440422,47.768733,69.777534,82.981462,106.712112
GS,3776.0,149.908232,40.793310,41.506359,123.869452,144.055862,180.055828,250.703659
BAC,3776.0,19.189126,9.695198,2.612258,11.788328,14.794664,26.732720,39.855698
BA,3776.0,127.800080,101.063701,22.244297,57.555450,87.499565,148.790710,430.299988
RTX,3776.0,47.553019,17.303090,16.908560,31.983736,46.883154,60.218807,91.033150
MSFT,3776.0,51.729165,48.376271,11.376367,20.866843,27.605119,60.466048,226.430359


In [37]:
# Save the Close Price datase
df_close.to_csv('datasets/close_prices.csv', index=False)